<a href="https://colab.research.google.com/github/FunmilolaAdetuwo/Final-Project/blob/main/Analyzing_the_Impact_of_Mergers_and_Acquisitions_on_Stock_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Let's install SEC Edgar downloader library firstg

!pip install sec-edgar-downloader


In [6]:
# Importing necessary libraries

import yfinance as yf
import pandas as pd
from datetime import datetime
import os
from sec_edgar_downloader import Downloader


Let's list the name of the companies for the different Merger and Acquisition(M&A) types

For the stock price years we will be using 5 year stock price, which will be 2.5 years pre M&A date and 2.5 years post M&A dates
the companies that will be used are: Microsoft which acquired LinkedIn (for Horizontal type of M&A)
Amazon which acquired Whole Foods (for Vertical type of M&A)
Berkshire Hathaway which acquired Precision Castparts (for Conglomerate type of M&A)

first let's extract the M&A dates from SEC Edgar

In [14]:
# Let's mount the drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# defining where the downloaded fillings will be saved in the directory
download_flgs = "/content/drive/My Drive/downloaded_filings"

In [17]:
# Checking that the directory exists
if not os.path.exists(download_flgs):
    os.makedirs(download_flgs)

To comply with the SEC's fair access rule, where a user of their electronic system must provide the email address and company name , we will now provide an email address as part of initialization of the downloader

In [28]:
compy_name = "University of Hertfordshire"
email_add = "afummite@gmail.com"

In [31]:
# Initializing the downloader
def download_sec_filings(ticker, num_filings):
    dl = Downloader(compy_name, email_add, download_folder=download_flgs)
    dl.get("8-K", ticker, limit=num_filings)


In [32]:
# The Companies and their tickers
cmpy_tick = {
    "MSFT": "Microsoft Corporation",
    "AMZN": "Amazon.com, Inc.",
    "BRK-A": "Berkshire Hathaway Inc."
}


for ticker in cmpy_tick.keys():
    download_sec_filings(ticker, 10)


In [33]:

def extract_ma_dates(ticker):
    return "YYYY-MM-DD"

In [35]:
# Storing the M&A dates in dictionary
ma_dates = {}

# Let's extract the M&A dates for the companies
for ticker in cmpy_tick.keys():
    ma_dates[ticker] = extract_ma_dates(ticker)

In [38]:
# Now displaying the M&A dates
print("M&A announcement dates:")
for ticker, date in ma_dates.items():
    print(f"{cmpy_tick[ticker]} ({ticker}): {date}")

M&A announcement dates:
Microsoft Corporation (MSFT): YYYY-MM-DD
Amazon.com, Inc. (AMZN): YYYY-MM-DD
Berkshire Hathaway Inc. (BRK-A): YYYY-MM-DD


In [ ]:
cmpys = {
    'horizontal': {'company': 'MSFT', 'ma_dte': '2016-06-13', 'start_date': '2013-12-13', 'end_date': '2018-12-13'},
    'vertical': {'company': 'AMZN', 'ma_dte': '2017-06-16', 'start_date': '2014-12-16', 'end_date': '2019-12-16'},
    'conglomerate': {'company': 'BRK.A', 'ma_dte': '2015-08-10', 'start_date': '2013-02-10', 'end_date': '2018-02-10'}
}


In [ ]:
# Download stock price data from Yahoo Finance for 2.5 years before and after
def download_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

# Collect stock data for each M&A type
stock_data = {}
for mna_type, company_info in companies.items():
    ticker = company_info['company']
    start_date = company_info['start_date']
    end_date = company_info['end_date']
    stock_data[mna_type] = download_stock_data(ticker, start_date, end_date)

# Combine with M&A dates
mna_dates = pd.DataFrame(companies).T
mna_dates['mna_date'] = pd.to_datetime(mna_dates['mna_date'])

# Display the stock data for validation
print(stock_data)